# 03 - Regularization mit Tensorflow

Nachdem wir uns den grundsätzlichen Aufbau eines neuronalen Netzes angesehen haben wollen wir uns ansehen, wie wir mittels Regularisierung das Lernen verbessern können.

In [1]:
import tensorflow as tf

2024-01-13 22:30:09.476507: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-01-13 22:30:09.477724: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-01-13 22:30:09.495491: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-13 22:30:09.495515: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-13 22:30:09.495527: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to regi

In [2]:
%matplotlib inline

In [3]:
print(tf.__version__)

2.14.0


## Erstellen eines kleinen klassifikationsdaten

In [4]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical

iris = datasets.load_iris()
y = to_categorical(iris.target)
x = iris.data
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)

In [5]:
from keras import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
from keras.losses import CategoricalCrossentropy
from keras.metrics import CategoricalAccuracy, Recall, Precision

model = Sequential(name="classification_mlp")
model.add(Dense(8, activation='relu', input_dim=4))
model.add(Dense(3, activation='softmax'))

model.compile(
    optimizer=Adam(learning_rate=0.01),
    loss=CategoricalCrossentropy(),
    metrics=[CategoricalAccuracy(), Recall(), Precision()]
)

model.fit(
    x=x_train, 
    y=y_train,
    validation_split=0.2,
    epochs=5,
)
loss, acc, rec, prec = model.evaluate(x_test, y_test)
tf.keras.backend.clear_session()

Epoch 1/5


2024-01-13 22:30:10.934440: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-01-13 22:30:10.934585: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2211] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


3/3 [==============================] - 1s 63ms/step - loss: 3.5397 - categorical_accuracy: 0.3214 - recall: 0.3214 - precision: 0.3214 - val_loss: 2.9498 - val_categorical_accuracy: 0.1905 - val_recall: 0.1905 - val_precision: 0.1905
Epoch 2/5
3/3 [==============================] - 0s 9ms/step - loss: 2.3659 - categorical_accuracy: 0.3214 - recall: 0.3214 - precision: 0.3214 - val_loss: 1.6706 - val_categorical_accuracy: 0.1905 - val_recall: 0.1905 - val_precision: 0.1905
Epoch 3/5
3/3 [==============================] - 0s 9ms/step - loss: 1.5224 - categorical_accuracy: 0.4286 - recall: 0.2619 - precision: 0.4000 - val_loss: 1.1712 - val_categorical_accuracy: 0.4762 - val_recall: 0.4762 - val_precision: 0.5882
Epoch 4/5
3/3 [==============================] - 0s 9ms/step - loss: 1.1891 - categorical_accuracy: 0.3571 - recall: 0.3214 - precision: 0.4737 - val_loss: 0.9501 - val_categorical_accuracy: 0.4762 - val_recall: 0.4762 - val_precision: 0.5882
Epoch 5/5
2/2 [======================

## Regularization

Regularisierung wird verwendet, um Overfittig beim Trainieren eines Models entgegenzuwirken, sowie die Komplexität in einem neuronalen Netz zu reduzieren. Dabei können Early Stopping, Weight Constraining, Weight Penalizing und Droupout verwendet werden. Die letzten beiden davon können ebenso zur Reduktion der komplexität des Netzes verwendet werden.

Methoden zur Regularisierung:
* Early Stopping
* Weight oriented:
    * Weight Penalizing: L1 (Lasso), L2 (Ridge), L1L2 (ElasticNet)
    * Weight Constraining: Unit Norm, Max Norm, Min-Max Norm
* Network oriented:
    * Network agnostic: Vanilla Dropout, Gaussian Dropout, Stand-out

### Weight oriented
Hierbei wird auf die Gewichte im Kernel oder Bias des jeweiligen Layers Einfluss genommen.

In [6]:
from keras.regularizers import L1
from keras.constraints import MaxNorm

model = Sequential(name="classification_mlp")
model.add(Dense(512, activation='relu', input_dim=4, kernel_regularizer=L1(l1=0.01), kernel_constraint=MaxNorm(1.5)))
model.add(Dense(3, activation='softmax'))

model.compile(
    optimizer=Adam(learning_rate=0.01),
    loss=CategoricalCrossentropy(),
    metrics=[CategoricalAccuracy(), Recall(), Precision()]
)

model.fit(
    x=x_train, 
    y=y_train,
    validation_split=0.2,
    epochs=5,
)
loss, acc, rec, prec = model.evaluate(x_test, y_test)
tf.keras.backend.clear_session()

Epoch 1/5
3/3 [==============================] - 0s 56ms/step - loss: 2.2489 - categorical_accuracy: 0.3333 - recall: 0.3333 - precision: 0.3457 - val_loss: 1.5546 - val_categorical_accuracy: 0.5238 - val_recall: 0.4762 - val_precision: 0.5882
Epoch 2/5
3/3 [==============================] - 0s 9ms/step - loss: 1.5098 - categorical_accuracy: 0.5714 - recall: 0.4762 - precision: 0.6349 - val_loss: 1.5653 - val_categorical_accuracy: 0.2381 - val_recall: 0.1905 - val_precision: 0.5714
Epoch 3/5
3/3 [==============================] - 0s 8ms/step - loss: 1.2406 - categorical_accuracy: 0.7024 - recall: 0.5476 - precision: 0.7931 - val_loss: 1.1390 - val_categorical_accuracy: 0.6667 - val_recall: 0.6667 - val_precision: 0.6667
Epoch 4/5
3/3 [==============================] - 0s 9ms/step - loss: 1.1687 - categorical_accuracy: 0.6310 - recall: 0.6190 - precision: 0.6265 - val_loss: 1.0923 - val_categorical_accuracy: 0.5714 - val_recall: 0.5714 - val_precision: 0.5714
Epoch 5/5
2/2 [============

### Network oriented
Hierbei wird auf die Architektur bzw. Verbindungen der bzw zwischen den Layers Einfluss genommen.

In [7]:
from keras.layers import Dropout

model = Sequential(name="classification_mlp")
model.add(Dense(512, activation='relu', input_dim=4))
model.add(Dropout(0.2))
model.add(Dense(3, activation='softmax'))

model.compile(
    optimizer=Adam(learning_rate=0.01),
    loss=CategoricalCrossentropy(),
    metrics=[CategoricalAccuracy(), Recall(), Precision()]
)

model.fit(
    x=x_train, 
    y=y_train,
    validation_split=0.2,
    epochs=5,
)
loss, acc, rec, prec = model.evaluate(x_test, y_test)
tf.keras.backend.clear_session()

Epoch 1/5
3/3 [==============================] - 0s 55ms/step - loss: 1.1390 - categorical_accuracy: 0.4643 - recall: 0.3571 - precision: 0.5769 - val_loss: 0.7413 - val_categorical_accuracy: 0.6667 - val_recall: 0.6667 - val_precision: 0.6667
Epoch 2/5
3/3 [==============================] - 0s 9ms/step - loss: 0.8499 - categorical_accuracy: 0.6429 - recall: 0.5714 - precision: 0.6400 - val_loss: 0.6718 - val_categorical_accuracy: 0.5238 - val_recall: 0.5238 - val_precision: 0.5238
Epoch 3/5
3/3 [==============================] - 0s 9ms/step - loss: 0.6181 - categorical_accuracy: 0.6429 - recall: 0.6310 - precision: 0.6463 - val_loss: 0.5624 - val_categorical_accuracy: 0.6667 - val_recall: 0.6667 - val_precision: 0.6667
Epoch 4/5
3/3 [==============================] - 0s 9ms/step - loss: 0.5408 - categorical_accuracy: 0.7381 - recall: 0.6905 - precision: 0.7532 - val_loss: 0.5468 - val_categorical_accuracy: 0.5238 - val_recall: 0.5238 - val_precision: 0.5238
Epoch 5/5
2/2 [============

### Early Stopping
Hierbei wird das Training gestoppt so bald keine Verbesserung auf den Validierungsdaten erzeugt wird.

In [8]:
from keras.callbacks import EarlyStopping

model = Sequential(name="classification_mlp")
model.add(Dense(512, activation='relu', input_dim=4))
model.add(Dense(3, activation='softmax'))

model.compile(
    optimizer=Adam(learning_rate=0.1),
    loss=CategoricalCrossentropy(),
    metrics=[CategoricalAccuracy(), Recall(), Precision()]
)

model.fit(
    x=x_train, 
    y=y_train,
    validation_split=0.2,
    epochs=50,
    callbacks=[EarlyStopping()]
)
loss, acc, rec, prec = model.evaluate(x_test, y_test)
tf.keras.backend.clear_session()

Epoch 1/50
3/3 [==============================] - 0s 55ms/step - loss: 14.3014 - categorical_accuracy: 0.2619 - recall: 0.2500 - precision: 0.3559 - val_loss: 7.6951 - val_categorical_accuracy: 0.6667 - val_recall: 0.6667 - val_precision: 0.6667
Epoch 2/50
3/3 [==============================] - 0s 9ms/step - loss: 7.3094 - categorical_accuracy: 0.4405 - recall: 0.4405 - precision: 0.4405 - val_loss: 0.9391 - val_categorical_accuracy: 0.6190 - val_recall: 0.5238 - val_precision: 0.6111
Epoch 3/50
2/2 [==============================] - 0s 3ms/step - loss: 1.4292 - categorical_accuracy: 0.4222 - recall: 0.4222 - precision: 0.4222


---

Wahlpflichtach Künstliche Intelligenz II: Praktikum 